In [1]:
import numpy as np
from scipy.optimize import least_squares
#from pandas import Series, DataFrame
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
%matplotlib qt5
#
# if pade.py is not in the current directory, set this path:
#
#import sys
#sys.path.append('../Python_libs')
from rac_aux import *

In [2]:
Angs2Bohr=1.8897259886
au2eV=27.211386027
au2cm=219474.63068
#
#  files in the current directory do not need the path name
#
#df = pd.read_csv("/home/thomas/Python/StabPlots/Stab_data/1D_a0.2_b0_c0.14/crossing_1.dat", delim_whitespace=True)
df = pd.read_csv("sb_rac.csv")
#df = pd.read_csv("crossing_1.dat", delim_whitespace=True)

plot_it=False
if plot_it:
    plt.cla()
    plt.plot(df.l.values, df.E1.values, 'o-')
    plt.plot(df.l.values, df.E2.values, 'o-')
    plt.plot(df.l.values, df.E3.values, 'o-')
    plt.show()
df[:5]

,l,E1,E2,E3,E4
0,0.0,0.469219,1.365173,1.938043,3.271038
1,0.2,0.457846,1.314401,1.844238,3.242423
2,0.4,0.446073,1.251127,1.760926,3.215267
3,0.6,0.433754,1.173608,1.690292,3.189317
4,0.8,0.420648,1.082215,1.632448,3.164365


In [3]:
#
# put all negative E(lambda) points into the vectors: ls and Es
#
i_neg = np.argmin(abs(df.E1.values))
if df.E1[i_neg] > 0:
    i_neg += 1
ls = df.l.values[i_neg:]
print('N=',len(ls))
Es = df.E1.values[i_neg:]
if plot_it:
    plt.cla()
    plt.plot(df.l.values, df.E1.values, 'b-')
    plt.plot(df.l.values, df.E2.values, 'b-')
    plt.plot(df.l.values, df.E3.values, 'b-')
    plt.plot(ls, Es, 'o', color="orange")
    plt.show()

N= 89


In [38]:
#
#  So far, nm can be in [21, 31, 32, 42, 53]
#
nm=32

fun=pade_32_lsq
jac=pade_32j_lsq

In [39]:
#
#  kappas, kappa**2, and sigmas (weights = sigma**2)
#  least_squares() passes parg to each pade_nm function
#
k2s = -Es
ks  = np.sqrt(k2s)
sigmas = weights(len(Es), 'ones')
#sigmas = weights(len(Es), 'energy', E0=Es[11], Es=Es)
parg=(ks,k2s,ls,sigmas)

In [40]:
# start params depend on nm
p31_opt = [2.4022, 0.2713, 1.2813, 0.4543]
p42_opt = [2.3919, 0.2964, 1.3187, 1.3736,  0.29655, 0.5078]

E0 = linear_extra(ls,Es)
G0 = 0.2*E0
if nm == 21:
    p0s=[ls[0]] + guess(E0, G0)
elif nm == 31:
    p0s=[ls[0]] + guess(E0, G0) + [10]
elif nm == 32:
    p0s=[ls[0]] + guess(E0, G0) + [10] + [1]
elif nm == 42:
    p0s=[ls[0]] + guess(E0, G0) + guess(5*E0,10*G0) + [10]
elif nm == 53:
    p0s = p42_opt[0:5] + p31_opt[3:] + p42_opt[5:] + [1]
else:
    print("Warning", nm, "not implemented")
print(p0s)
print(chi2_gen(p0s, ks, k2s, ls, sigmas, fun))

[2.4, 0.2704669890140844, 1.4666963639216455, 10, 1]
1.0015809965763467


In [41]:
#
#  test the derivative of [n,m] 
#

N=6

df1s = pade_gen_j_lsq(p0s, ks[-N:], k2s[-N:], ls[-N:], sigmas[-N:], fun)
print("num grad:\n", df1s)

df2s = jac(p0s, ks[-N:], k2s[-N:], ls[-N:], sigmas[-N:])
print("ana grad:\n", df2s)

np.sqrt(np.sum(np.square(df1s-df2s)))

num grad:
 [[ 0.77080867  0.39021477 -2.13292562  0.00635328 -0.06124711]
 [ 0.77901782  0.39225394 -2.15903672  0.00645951 -0.06228522]
 [ 0.78723047  0.39427653 -2.18515483  0.00656635 -0.06332949]
 [ 0.79544659  0.39628284 -2.21127985  0.00667381 -0.0643799 ]
 [ 0.80366612  0.39827313 -2.2374117   0.00678189 -0.06543642]
 [ 0.81188905  0.40024768 -2.26355027  0.00689057 -0.06649903]]
ana grad:
 [[ 0.77080867  0.39021477 -2.13292562  0.00635328 -0.06124711]
 [ 0.77901782  0.39225394 -2.15903672  0.00645951 -0.06228522]
 [ 0.78723047  0.39427653 -2.18515483  0.00656635 -0.06332949]
 [ 0.79544659  0.39628284 -2.21127985  0.00667381 -0.0643799 ]
 [ 0.80366612  0.39827313 -2.2374117   0.00678189 -0.06543642]
 [ 0.81188905  0.40024768 -2.26355027  0.00689057 -0.06649903]]


1.1290223162891018e-09

In [42]:
print('Least squares, trust-region-reflective (default) with 2-point jac')
res = least_squares(fun, p0s, method='trf', jac='2-point', args=parg)
print("njev:",res.njev)
print("cost:",res.cost)
print("grad:",res.grad)
print("message:",res.message)
print("success:",res.success)
print("x:", res.x)
print('chi2 = %.3e' % (res.cost*2))
print("Er=%f,  Gamma=%f" % res_ene(res.x[1], res.x[2]))

Least squares, trust-region-reflective (default) with 2-point jac
njev: 98
cost: 3.668166195430488e-08
grad: [-2.48509482e-11 -3.05390182e-12  5.03193369e-11 -1.27157436e-11
  1.97579929e-11]
message: `gtol` termination condition is satisfied.
success: True
x: [2.39286302 0.31720262 1.31445718 1.03224749 0.18596322]
chi2 = 7.336e-08
Er=1.717674,  Gamma=0.529030


In [43]:
print('Least squares, trust-region-reflective (default) with analytic jac')
res = least_squares(fun, p0s, method='trf', jac=jac, args=parg)
print("njev:",res.njev)
print("cost:",res.cost)
print("grad:",res.grad)
print("message:",res.message)
print("success:",res.success)
print("x:", res.x)
print('chi2 = %.3e' % (res.cost*2))
print("Er=%f,  Gamma=%f" % res_ene(res.x[1], res.x[2]))

Least squares, trust-region-reflective (default) with analytic jac
njev: 98
cost: 3.6681661954263234e-08
grad: [-2.63597878e-11  1.25826368e-11  7.03996035e-11 -1.03581419e-11
  2.80148887e-11]
message: `gtol` termination condition is satisfied.
success: True
x: [2.39286302 0.31720262 1.31445718 1.03224742 0.1859632 ]
chi2 = 7.336e-08
Er=1.717674,  Gamma=0.529030


In [44]:
print('Least squares, Levenberg-Marquardt with analytic jac')
res = least_squares(fun, p0s, method='lm', jac=jac, args=parg)
print("njev:",res.njev)
print("cost:",res.cost)
print("grad:",res.grad)
print("message:",res.message)
print("success:",res.success)
print("x:", res.x)
print('chi2 = %.3e' % (res.cost*2))
print("Er=%f,  Gamma=%f" % res_ene(res.x[1], res.x[2]))

Least squares, Levenberg-Marquardt with analytic jac
njev: 92
cost: 3.668166195416088e-08
grad: [-2.77802261e-11  1.32567909e-11  7.41961591e-11 -1.09292853e-11
  2.95169864e-11]
message: `ftol` termination condition is satisfied.
success: True
x: [2.39286302 0.31720262 1.31445718 1.03224742 0.1859632 ]
chi2 = 7.336e-08
Er=1.717674,  Gamma=0.529030


In [45]:
print('Least squares, TRF with bounds')

#
#  bnds depend on the number of parameters
#
npara=len(p0s)
zs = np.zeros(npara)
infs = np.full(npara, np.inf)
bnds=(zs, infs)

res = least_squares(fun, p0s, jac=jac, bounds=bnds, args=parg)
print("njev:",res.njev)
print("cost:",res.cost)
print("grad:",res.grad)
print("message:",res.message)
print("success:",res.success)
print("x:", res.x)
print('chi2 = %.3e' % (res.cost*2))
print("Er=%f,  Gamma=%f" % res_ene(res.x[1], res.x[2]))

Least squares, TRF with bounds
njev: 31
cost: 2.6744702809644768e-06
grad: [-8.12658327e-10 -4.76519391e-10  2.22819483e-09 -7.83010650e-12
  1.58066482e-10]
message: `gtol` termination condition is satisfied.
success: True
x: [ 2.37126798  0.27344434  1.33872887 33.01593234  6.70979226]
chi2 = 5.349e-06
Er=1.786604,  Gamma=0.400397


In [46]:
#
# swipe energy filter 
#
M=len(Es)
sigmas = weights(M, 'ones')
res = least_squares(fun, p0s, method='trf', jac=jac, args=(ks, k2s, ls, sigmas))
Er, G = res_ene(res.x[1], res.x[2])
print('All weights equal:  chi2 = %.3e  Er=%f,  Gamma=%f' % (res.cost*2, Er, G))
M=len(Es)
for n in [0, M//4, M//2, 3*M//4, M-1]:
    sigmas = weights(M, 'energy', E0=Es[n], Es=Es)
    res = least_squares(fun, p0s, method='trf', jac=jac, args=(ks, k2s, ls, sigmas))
    Er, G = res_ene(res.x[1], res.x[2])
    print('Filter E = %6.2f:  chi2 = %.3e  Er=%f,  Gamma=%f' % (Es[n], res.cost*2, Er, G))

All weights equal:  chi2 = 7.336e-08  Er=1.717674,  Gamma=0.529030
Filter E =  -0.00:  chi2 = 4.824e-08  Er=1.727520,  Gamma=0.521586
Filter E =  -3.30:  chi2 = 2.607e-08  Er=1.703503,  Gamma=0.510255
Filter E =  -6.81:  chi2 = 2.007e-08  Er=1.693914,  Gamma=0.501228
Filter E = -10.43:  chi2 = 1.762e-08  Er=1.687185,  Gamma=0.492727
Filter E = -14.11:  chi2 = 1.899e-08  Er=1.678523,  Gamma=0.477630
